# Uebung 03

Gruppe: Josef Koch, Thomas Wally

## Anmerkungen

Für diese Übung ist es zugelassen, dass Sie in __Gruppen von zwei Personen__ arbeiten.
Sie dürfen auch alleine arbeiten wenn Sie das bevorzugen.
Beide Gruppenmitglieder müssen die Kreuzerlliste ausfüllen und die Ausarbeitung hochladen.
Auch müssen die Namen beider Gruppenmitglieder in der Ausarbeitung vermerkt werden.

Geben Sie bitte Quellen an wenn Sie Code aus dem Internet Kopieren.
Laden Sie Ihre Lösung nur in privaten Repos auf Platformen wie GitHub, GitLab, usw. hoch.

## Decision Tree und Random Forest

In dieser Uebung sollen Sie einen Decision Tree mit Hilfe des ID3-Algorithmus selbst implementieren.
Die Performance ihrer Implementierung sollen Sie mit der Implementierung einer Bibliothek vergleichen.
Im Anschluss sollen Sie die Performance des Decision Trees mit der des Random Forest vergleichen.
Zur Klassifizierung verwenden wir dieses Mal den [Breast Cancer Datensatz](https://archive.ics.uci.edu/ml/datasets/Breast+Cancer).
Die unten angegebenen Files `breast-cancer_train.data` und `breast-cancer_test.data` finden Sie im Moodle.

### Aufgabe 03.01 (15 Punkte)

Implementieren Sie selbststaendig einen Decision Tree.
Verwenden Sie dazu den ID3-Algorithmus.

- [x] Daten einlesen
- [x] Decision Tree implementieren
- [x] Decision Tree erstellen (trainieren mit `breast-cancer_train.data`)
- [x] Accuracy mit `breast-cancer_test.data` berechnen
- [x] Laufzeit des Training des Decision Tree angeben

Sie muessen kein Pruning fuer den Baum implementieren.
Sie muessen fuer diese Aufgabe keine _k_-Fold-Cross-Validation machen.

Implementierungshinweise:
* Fuer die Berechnung des Information Gain duerfen Sie Funktionen aus der `math` oder `numpy` Bibliotek verwenden.
* Sie duerfen __pandas__(`pandas`) verwenden um die Daten einzulesen und zu speichern.
* Sie duerfen __scikit-learn__(`sklearn`) fuer die Berechnung der Accuracy verwenden.
* Sie duerfen fuer den Decision Tree __keine__ Machine Learning Bibliothek verwenden (ausgenommen der oben angefuehrten).

### Aufgabe 03.02 (5 Punkte)

Verwenden Sie eine fertige Implementierung des Decision Trees von __scikit-learn__(`sklearn`).

- [x] Daten einlesen
- [x] Decision Tree mit __scikit-learn__(`sklearn`) erstellen (trainieren mit `breast-cancer_train.data`)
- [x] Accuracy mit `breast-cancer_test.data` berechnen
- [x] Verlgeichen Sie die Accuracy Ihrer Implementierung der des Decision Tree den Sie mit __scikit-learn__ erstellt haben. Versuchen Sie den Unterschied in der Accuracy zu erklaeren.

Implementierungshinweise:
* Sie duerfen __pandas__(`pandas`) verwenden um die Daten einzulesen und zu speichern.
* Sie duerfen __scikit-learn__(`sklearn`) fuer die Berechnung der Accuracy verwenden.
* Sie sollen __scikit-learn__(`sklearn`) fuer den Decision Tree verwenden.

### Aufgabe 03.03 (5 Bonuspunkte)

Verwenden Sie eine fertige Implementierung des Random Forest von __scikit-learn__(`sklearn`).

- [x] Daten einlesen
- [x] Random Forest mit __scikit-learn__(`sklearn`) erstellen. Tunen Sie die Hyperparameter mit _k_-Fold-Cross-Validation mit `breast-cancer_train.data`.
- [x] Accuracy mit `breast-cancer_test.data` berechnen
- [x] Verlgeichen Sie die Accuracy des Decision Tree den Sie mit erstellt haben __scikit-learn__ mit der des Random Forest den Sie mit __scikit-learn__ erstellt haben. Versuchen Sie den Unterschied in der Accuracy zu erklaeren.

Implementierungshinweise:
* Sie duerfen __pandas__(`pandas`) verwenden um die Daten einzulesen und zu speichern.
* Sie duerfen __scikit-learn__(`sklearn`) fuer die Berechnung der Accuracy verwenden.
* Sie duerfen __scikit-learn__(`sklearn`) fuer _k_-Fold-Cross-Validation verwenden.
* Sie sollen __scikit-learn__(`sklearn`) fuer den Random Forest verwenden.

### Anmerkungen

Vergessen Sie nicht ihre Abgabe entsprechend zu dokumentieren.

Laden Sie die von Ihnen bearbeitete `.ipynb` Datei sowie ein dazugehoeriges `requirements.txt` File in einer `.zip` Datei im Moodle Kurs hoch.
Achten Sie darauf, dass keine Umlaute im Namen ihrer abgegebenen Files enthalten sind.

---
#### Data Features
1. Class: no-recurrence-events, recurrence-events
2. age: 10-19, 20-29, 30-39, 40-49, 50-59, 60-69, 70-79, 80-89, 90-99.
3. menopause: lt40, ge40, premeno.
4. tumor-size: 0-4, 5-9, 10-14, 15-19, 20-24, 25-29, 30-34, 35-39, 40-44, 45-49, 50-54, 55-59.
5. inv-nodes: 0-2, 3-5, 6-8, 9-11, 12-14, 15-17, 18-20, 21-23, 24-26, 27-29, 30-32, 33-35, 36-39.
6. node-caps: yes, no.
7. deg-malig: 1, 2, 3.
8. breast: left, right.
9. breast-quad: left-up, left-low, right-up, right-low, central.
10. irradiat: yes, no.

### Aufgabe 03.01

In [146]:
import pandas as pd
import math
from sklearn.metrics import accuracy_score
import timeit
import pprint



# Calculate the entropy of a given data frame column
# Lecture03.pdf, page 34
def calc_entropy(col):
    values = col.unique()
    ent = 0
    for value in values:
        # Amount of a specific value / total amount of values
        pi = col.value_counts()[value] / len(col)
        ent += - pi * math.log2(pi)
    return ent


# Calculate the information gain of a given data frame and attribute
# Lecture03.pdf, page 36
def calc_gain(df, attribute, target_attribute = "class"):
    total_ent = calc_entropy(df[target_attribute])
    attribute_values = df[attribute].unique()
    attribute_ent = 0
    for value in attribute_values:
        value_df = df.loc[df[attribute] == value]
        entropy_df = calc_entropy(value_df[target_attribute])
        value_ratio = len(value_df) / len(df)
        attribute_ent += value_ratio * entropy_df
    return total_ent - attribute_ent


# Build the tree using the ID3 algorithm
# https://en.wikipedia.org/wiki/ID3_algorithm
def id3(df, attributes, target_attribute = "class"):
    
    # If there is only one value of the target attribute in the data frame, then return this value
    if len(df[target_attribute].unique()) == 1:
        return df[target_attribute].unique()[0]
    
    # If the attributes are empty, then return most common value of the target attribute in the data frame
    if len(attributes) == 0:
        return df[target_attribute].mode()[0]
    
    # Get the attribute with the highest information gain
    gains = {}
    for attribute in attributes:
        gains[attribute] = calc_gain(df, attribute)
    best_attribute = max(gains, key=gains.get)
    
    # Build the tree
    tree = { best_attribute: {} }
    
    # Build subtree for each attribute value
    attribute_values = df[best_attribute].unique()
    for att_value in attribute_values:
        att_df = df.loc[df[best_attribute] == att_value]
        
        # If the new data frame is empty,
        # the subtree is simply the most common value of the target attribute in the data frame
        if len(att_df) == 0:
            subtree = df[target_attribute].mode()[0]
        else:
            # Remove the best attribute from the attributes array
            attributes = [i for i in attributes if i != best_attribute]  
    
            # Recursively call the function again with the new data frame and updated attributes
            subtree = id3(att_df, attributes)
        
        # Now add the subtree to the above created tree
        tree[best_attribute][att_value] = subtree
    
    return tree


# Prediction based on a given query {"attribute_name_1": "attribute_value", ...}, tree and default value
# Inspiration: https://www.python-course.eu/Decision_Trees.php
def predict(query, tree, default):
    # Iterate through every attribute name of the query
    for key in list(query.keys()):
        # If the attribute name is in the list of values of the root node
        if key in list(tree.keys()):
            # Check if key and value from the query exist on the tree, return the default if not
            try:
                tree[key][query[key]] 
            except KeyError:
                return default
            
            result = tree[key][query[key]]
            
            # if the result is another dictionary (tree) we call the function again (recursive)
            if isinstance(result,dict):
                return predict(query, result, default)
            
            # if not we return the result (no-recurrence-events or recurrence-events)
            else:
                return result
            
       
    
breast_features = ["class", "age", "menopause", "tumor-size", "inv-nodes", "node-caps", "deg-malig", "breast", "breast-quad", "irradiat"]
breast_train = pd.read_csv("breast-cancer_train.data", names=breast_features)
breast_test = pd.read_csv("breast-cancer_test.data", names=breast_features)

start = timeit.default_timer()

# Calculate the tree as a dictionary with the ID3 algorithm by providing the data frame and features without "class"
breast_train_tree = id3(breast_train, breast_features[1:])

end = timeit.default_timer()

# The most common value of the target attribute in the data frame
breast_test_default = breast_test["class"].mode()[0]

# Create a dictionary from the breast test data without the "class" column
breast_test_dict = breast_test.iloc[:, 1:].to_dict(orient = "records")

y_true = breast_test["class"].tolist()
y_pred = []

for breast_query in breast_test_dict:
    y_pred.append(predict(breast_query, breast_train_tree, breast_test_default))

print("Accuracy = %.2f%%" % (accuracy_score(y_true, y_pred)*100))
print("Time = %.2fs" % (end - start))
#pprint.pprint(breast_train_tree)



Accuracy = 66.23%
Time = 6.02s


---
### Aufgabe 03.02

Die Accuracy des sklearn Descion Trees ist deshalb schlechter weil hier zahlreiche optimierungen eingesetzt werden
die zwar eine sehr kurze Laufzeit mit sich bringen, allerdings zur Folge haben das die Accuracy etwas darunter leidet.
Auch sind hier die standard Werte des Decision Tree Classifier benutzt worden die natürlich auch nicht optimal sind.

In [147]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import timeit



# Encode the dataf rame values which are strings to integers to use them in the DecisionTreeClassifier from sklearn
# Inspiration: http://chrisstrelioff.ws/sandbox/2015/06/08/decision_trees_in_python_with_scikit_learn_and_pandas.html
def encode_dataframes(df_train, df_test, features):
    feature_vals = {
        "class": ["no-recurrence-events", "recurrence-events"], 
        "age": ["10-19", "20-29", "30-39", "40-49", "50-59", "60-69", "70-79", "80-89", "90-99"],
        "menopause": ["lt40", "ge40", "premeno"],
        "tumor-size": ["0-4", "5-9", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40-44", "45-49", "50-54", "55-59"],
        "inv-nodes": ["0-2", "3-5", "6-8", "9-11", "12-14", "15-17", "18-20", "21-23", "24-26", "27-29", "30-32", "33-35", "36-39"],
        "node-caps": ["yes", "no"],
        "deg-malig": [1, 2, 3],
        "breast": ["left", "right"],
        "breast-quad": ["left_up", "left_low", "right_up", "right_low", "central"],
        "irradiat": ["yes", "no"]
    }
    
    df_train_mod = df_train.copy()
    df_test_mod = df_test.copy()
    
    for feature in features:
        # create a dictionary to map the string values (name) to their corresponding index in the array 
        map_to_int = {name: n for n, name in enumerate(feature_vals[feature])}
        # replace the values of the train and test data frame with the map dictionary accordingly
        df_train_mod[feature] = df_train_mod[feature].replace(map_to_int)
        df_test_mod[feature] = df_test_mod[feature].replace(map_to_int)  
        
    return df_train_mod, df_test_mod



breast_features = ["class", "age", "menopause", "tumor-size", "inv-nodes", "node-caps", "deg-malig", "breast", "breast-quad", "irradiat"]
breast_train = pd.read_csv("breast-cancer_train.data", names=breast_features)
breast_test = pd.read_csv("breast-cancer_test.data", names=breast_features)

encoded_breast_train, encoded_breast_test = encode_dataframes(breast_train, breast_test, breast_features)

y_train = encoded_breast_train["class"]
y_test = encoded_breast_test["class"]
X_train = encoded_breast_train[breast_features[1:]]
X_test = encoded_breast_test[breast_features[1:]]

start = timeit.default_timer()
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)
end = timeit.default_timer()
y_pred = dt.predict(X_test)

print("Accuracy = %.2f%%" % (accuracy_score(y_test, y_pred)*100))
print("Time = %.2fs" % (end - start))


Accuracy = 63.64%
Time = 0.02s


---
### Aufgabe 03.03

Natürlich hat der Random Forest mit hyper parameter tuning die beste Accuracy aber auch weil ein Random Forest
sehr gute Ergebnisse mit einer kleinen Anzahl von Samples liefert und sich deshalb recht gut für unseren 
Trainingsdatensatz eignet. 

In [148]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
import numpy as np
import timeit



# Same as in Aufgabe 03.02
def encode_dataframes(df_train, df_test, features):
    feature_vals = {
        "class": ["no-recurrence-events", "recurrence-events"], 
        "age": ["10-19", "20-29", "30-39", "40-49", "50-59", "60-69", "70-79", "80-89", "90-99"],
        "menopause": ["lt40", "ge40", "premeno"],
        "tumor-size": ["0-4", "5-9", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40-44", "45-49", "50-54", "55-59"],
        "inv-nodes": ["0-2", "3-5", "6-8", "9-11", "12-14", "15-17", "18-20", "21-23", "24-26", "27-29", "30-32", "33-35", "36-39"],
        "node-caps": ["yes", "no"],
        "deg-malig": [1, 2, 3],
        "breast": ["left", "right"],
        "breast-quad": ["left_up", "left_low", "right_up", "right_low", "central"],
        "irradiat": ["yes", "no"]
    }
    
    df_train_mod = df_train.copy()
    df_test_mod = df_test.copy()
    
    for feature in features:
        # Create a dictionary to map the string values (name) to their corresponding index in the array 
        map_to_int = {name: n for n, name in enumerate(feature_vals[feature])}
        # Replace the values of the train and test data frame with the map dictionary accordingly
        df_train_mod[feature] = df_train_mod[feature].replace(map_to_int)
        df_test_mod[feature] = df_test_mod[feature].replace(map_to_int)  
        
    return df_train_mod, df_test_mod


# Source: https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74
def random_tuning(X_train, y_train):
    rf = RandomForestRegressor(random_state = 42)
    # Number of trees in random forest
    n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
    # Number of features to consider at every split
    max_features = ["auto", "sqrt"]
    # Maximum number of levels in tree
    max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
    max_depth.append(None)
    # Minimum number of samples required to split a node
    min_samples_split = [2, 5, 10]
    # Minimum number of samples required at each leaf node
    min_samples_leaf = [1, 2, 4]
    # Method of selecting samples for training each tree
    bootstrap = [True, False]# Create the random grid
    random_grid = {"n_estimators": n_estimators,
                   "max_features": max_features,
                   "max_depth": max_depth,
                   "min_samples_split": min_samples_split,
                   "min_samples_leaf": min_samples_leaf,
                   "bootstrap": bootstrap}

    # Use the random grid to search for best hyperparameters
    # First create the base model to tune
    rf = RandomForestRegressor()
    # Random search of parameters, using 3 fold cross validation, 
    # search across 100 different combinations, and use all available cores
    rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)# Fit the random search model
    rf_random.fit(X_train, y_train)
    rf_random.best_params_
    
    # Results
    """
    {"n_estimators": 400,
     "min_samples_split": 2,
     "min_samples_leaf": 4,
     "max_features": "sqrt",
     "max_depth": 10,
     "bootstrap": True}
    """

# Source: https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74 
def grid_search_tuning(X_train, y_train):
    # Create the parameter grid based on the results of random search 
    # Based on the results of the random tuning
    param_grid = {
        'bootstrap': [True],
        'max_depth': [3, 5, 10, 20],
        'max_features': [2, 5, 10],
        'min_samples_leaf': [2, 4, 7],
        'min_samples_split': [2, 4],
        'n_estimators': [200, 400, 500, 700]
    }# Create a based model
    rf = RandomForestRegressor()# Instantiate the grid search model
    grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, cv = 3, n_jobs = -1, verbose = 2)
    grid_search.fit(X_train, y_train)
    print(grid_search.best_params_)
    
    # Results
    """
    {'bootstrap': True, 
    'max_depth': 5, 
    'max_features': 2, 
    'min_samples_leaf': 4, 
    'min_samples_split': 4, 
    'n_estimators': 500}
    """



breast_features = ["class", "age", "menopause", "tumor-size", "inv-nodes", "node-caps", "deg-malig", "breast", "breast-quad", "irradiat"]
breast_train = pd.read_csv("breast-cancer_train.data", names=breast_features)
breast_test = pd.read_csv("breast-cancer_test.data", names=breast_features)

encoded_breast_train, encoded_breast_test = encode_dataframes(breast_train, breast_test, breast_features)

y_train = encoded_breast_train["class"]
y_test = encoded_breast_test["class"]
X_train = encoded_breast_train[breast_features[1:]]
X_test = encoded_breast_test[breast_features[1:]]

# Random hyper parameter tuning with 3 fold cross validation
# takes about 2 minutes on my laptop (i5, 4 threads)
#random_tuning(X_train, y_train)

# Grid hyper parameter tuning with 3 fold cross validation
# also takes about 2 minutes on my laptop (i5, 4 threads)
#grid_search_tuning(X_train, y_train)

start = timeit.default_timer()
# Use the Results from the grid search tuning
rf = RandomForestClassifier(n_estimators=500, min_samples_split=4, min_samples_leaf=4, max_features=2, max_depth=5, bootstrap=True)
rf.fit(X_train, y_train)
end = timeit.default_timer()
y_pred = rf.predict(X_test)

print("Accuracy = %.2f%%" % (accuracy_score(y_test, y_pred)*100))
print("Time = %.2fs" % (end - start))


Accuracy = 71.43%
Time = 2.10s
